In [38]:
import os
import csv
import pandas as pd
import numpy as np
import string
import re
import nltk
from sklearn import pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

#Visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

In [39]:
# ------ Data Text --------
# csv Data name
Data_sampel = ('E:\Program\[2] Program\AnSent\Data\Data_2_sampel\Indihome_all.csv')
Data_rem = ('E:\Program\[2] Program\AnSent\Data\Data_2_S_Remove\Rem_indihome.csv') #remove duppilcate
Data_prepro = ('E:\Program\[2] Program\AnSent\Data\Data_4_prepro\Pre_Indihome.csv')
# Data_prepro_2 = ('Data\Data_4_Prepro_rem\Indihome.csv') 
Data_lex = ('E:\Program\[2] Program\AnSent\Data\Data_5_Lex_sen\Lex_Indihome.csv')

In [40]:
#data1 = data input
data1 = pd.read_csv(Data_sampel, sep=(','))
data1.head()


,Unnamed: 0,Unnamed: 0.1,Tanggal,User ID,Data Teks
0,12473,12473,2021-10-27 07:31:26+00:00,arifsaadil,"@IndiHomeCare Sudah dilakukan sesuai DM, belum..."
1,9030,9030,2021-10-11 10:42:41+00:00,tojizru,@chifoeyuu P @IndiHome
2,8294,8294,2021-10-30 05:33:15+00:00,IndiHomeCare,@DilaPong Kak Dila. Uci cek updatenya dijadwal...
3,9506,9506,2021-10-09 12:36:33+00:00,lalafindyou_,@IndiHome ni kapan ya wifinya bener. Aku mau n...
4,3036,3036,2021-10-23 11:43:16+00:00,Razyajii,"Oy Mola TV di Indihome nape tak bisa dibuka, a..."


In [41]:
########################### REMOVE DUPLICATE ###########################
#################################################################

# #sort by name or number
# data1.sort_values('Data Teks_Stopword', ascending=False)

# jumlah data awal
index = data1.index
jumlah = len(index)

print('Jumlah data awal:',jumlah)

#jumlah data yang sama
data_sama = data1.duplicated(subset= "Data Teks").sum()
print('Jumlah data yang sama:',data_sama)

#remove duplicate data
remove_data = data1.drop_duplicates(subset = 'Data Teks', keep = 'first', inplace = True)
index = data1.index
jumlah = len(index)

print('Jumlah data sekarang:',jumlah)
data1.to_csv(Data_rem)

Jumlah data awal: 8000
Jumlah data yang sama: 211
Jumlah data sekarang: 7789


In [42]:
########################### CASEFOLDING ###########################
#################################################################

# ------ Proses CaseFolding --------
#------ Clean Text --------
#casefolding
data1['Data Teks_CaseFolding'] = data1['Data Teks'].str.lower()

def clean_text(tweet_clean):
  tweet_clean = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(tweet_clean)).split()) #mengghilangkan data teks (@, # dan link)
  tweet_clean = re.sub(r'\b\w{1,2}\b', '', tweet_clean) #menghilangkan 2 kata
  tweet_clean = re.sub('\s+',' ',tweet_clean)
  tweet_clean = re.sub(r"\d+", "", tweet_clean)
  #tweet_clean = removeDupWithOrder(tweet_clean) #mengghilangkan kata duplikat
  tweet_clean = tweet_clean.translate(str.maketrans("","",string.punctuation))# ini untuk apa 
  return tweet_clean
data1['Data Teks_CaseFolding'] = data1['Data Teks_CaseFolding'].apply(lambda x: clean_text(x))
data1.head()

,Unnamed: 0,Unnamed: 0.1,Tanggal,User ID,Data Teks,Data Teks_CaseFolding
0,12473,12473,2021-10-27 07:31:26+00:00,arifsaadil,"@IndiHomeCare Sudah dilakukan sesuai DM, belum...",sudah dilakukan sesuai belum bisa
1,9030,9030,2021-10-11 10:42:41+00:00,tojizru,@chifoeyuu P @IndiHome,
2,8294,8294,2021-10-30 05:33:15+00:00,IndiHomeCare,@DilaPong Kak Dila. Uci cek updatenya dijadwal...,kak dila uci cek updatenya dijadwalkan kembali...
3,9506,9506,2021-10-09 12:36:33+00:00,lalafindyou_,@IndiHome ni kapan ya wifinya bener. Aku mau n...,kapan wifinya bener aku mau nugas hikdddd pad...
4,3036,3036,2021-10-23 11:43:16+00:00,Razyajii,"Oy Mola TV di Indihome nape tak bisa dibuka, a...",mola indihome nape tak bisa dibuka appsnya il...


In [43]:
########################### NORMALIZE ###########################
#################################################################

alay_dict = pd.read_csv(r'E:\Program\[2] Program\AnSent\Dict_Used\kamus_baku.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})
alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])

data1['Data Teks_clean_alay'] = data1['Data Teks_CaseFolding'].apply(normalize_alay) 
data1.drop_duplicates(keep=False,inplace=True)

In [44]:
########################### TOKENIZING ###########################
#################################################################

# ------ Proses Tokenizing --------
def token(tweet_clean):
  tweet_clean = re.split('\W+', tweet_clean)
  return tweet_clean

data1['Data Teks_Token'] = data1['Data Teks_clean_alay'].apply(lambda x: token(x))
data1.head()

,Unnamed: 0,Unnamed: 0.1,Tanggal,User ID,Data Teks,Data Teks_CaseFolding,Data Teks_clean_alay,Data Teks_Token
0,12473,12473,2021-10-27 07:31:26+00:00,arifsaadil,"@IndiHomeCare Sudah dilakukan sesuai DM, belum...",sudah dilakukan sesuai belum bisa,sudah dilakukan sesuai belum bisa,"[sudah, dilakukan, sesuai, belum, bisa]"
1,9030,9030,2021-10-11 10:42:41+00:00,tojizru,@chifoeyuu P @IndiHome,,,[]
2,8294,8294,2021-10-30 05:33:15+00:00,IndiHomeCare,@DilaPong Kak Dila. Uci cek updatenya dijadwal...,kak dila uci cek updatenya dijadwalkan kembali...,kak dila uci periksa updatenya dijadwalkan ke...,"[kak, dila, uci, periksa, updatenya, dijadwalk..."
3,9506,9506,2021-10-09 12:36:33+00:00,lalafindyou_,@IndiHome ni kapan ya wifinya bener. Aku mau n...,kapan wifinya bener aku mau nugas hikdddd pad...,kapan wifinya benar aku mau nugas hikdddd pa...,"[, kapan, wifinya, benar, aku, mau, nugas, hik..."
4,3036,3036,2021-10-23 11:43:16+00:00,Razyajii,"Oy Mola TV di Indihome nape tak bisa dibuka, a...",mola indihome nape tak bisa dibuka appsnya il...,mola indihome nape tak bisa dibuka appsnya il...,"[, mola, indihome, nape, tak, bisa, dibuka, ap..."


In [45]:
########################### STOPWORD ###########################
#################################################################

nltk.download('stopwords')
from nltk.corpus import stopwords
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')
print(len(list_stopwords))

# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(['b',"yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah', 'kak'])
len(list_stopwords)
# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("E:\Program\[2] Program\AnSent\Dict_Used\stopwordsID.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
len(list_stopwords)
# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data1['Data Teks_Stopword'] = data1['Data Teks_Token'].apply(lambda x: stopwords_removal(x)) 


print(data1['Data Teks_Stopword'])

758
0                                                [sesuai]
1                                                      []
2       [dila, uci, periksa, updatenya, dijadwalkan, e...
3                 [, wifinya, nugas, hikdddd, dah, bayar]
4        [, mola, indihome, nape, dibuka, appsnya, ilang]
                              ...                        
7994    [, admin, koneksi, lost, koneksi, indikator, r...
7995    [halo, nana, bantu, respon, silakan, periksa, ...
7996    [halo, sobat, keluhan, seputar, produk, indiho...
7997    [hai, tettyrhs, maaf, terkait, informasi, kelu...
7999           [admin, digit, mohon, bantuannya, makasih]
Name: Data Teks_Stopword, Length: 7789, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
########################### STEMMING ###########################
#################################################################

stop_removal = data1['Data Teks_Stopword']

def fit_stopwords(tweet_clean):
    tweet_clean =np.array(tweet_clean)
    tweet_clean =' '.join(tweet_clean)
#    tweet_clean = tweet_clean.tosring()
#    tweet_clean = str(tweet_clean)
    return tweet_clean

data1['Data Teks_Stemming'] = data1['Data Teks_Stopword'].apply(lambda x: fit_stopwords(x))
data1.head(5)

,Unnamed: 0,Unnamed: 0.1,Tanggal,User ID,Data Teks,Data Teks_CaseFolding,Data Teks_clean_alay,Data Teks_Token,Data Teks_Stopword,Data Teks_Stemming
0,12473,12473,2021-10-27 07:31:26+00:00,arifsaadil,"@IndiHomeCare Sudah dilakukan sesuai DM, belum...",sudah dilakukan sesuai belum bisa,sudah dilakukan sesuai belum bisa,"[sudah, dilakukan, sesuai, belum, bisa]",[sesuai],sesuai
1,9030,9030,2021-10-11 10:42:41+00:00,tojizru,@chifoeyuu P @IndiHome,,,[],[],
2,8294,8294,2021-10-30 05:33:15+00:00,IndiHomeCare,@DilaPong Kak Dila. Uci cek updatenya dijadwal...,kak dila uci cek updatenya dijadwalkan kembali...,kak dila uci periksa updatenya dijadwalkan ke...,"[kak, dila, uci, periksa, updatenya, dijadwalk...","[dila, uci, periksa, updatenya, dijadwalkan, e...",dila uci periksa updatenya dijadwalkan esok ja...
3,9506,9506,2021-10-09 12:36:33+00:00,lalafindyou_,@IndiHome ni kapan ya wifinya bener. Aku mau n...,kapan wifinya bener aku mau nugas hikdddd pad...,kapan wifinya benar aku mau nugas hikdddd pa...,"[, kapan, wifinya, benar, aku, mau, nugas, hik...","[, wifinya, nugas, hikdddd, dah, bayar]",wifinya nugas hikdddd dah bayar
4,3036,3036,2021-10-23 11:43:16+00:00,Razyajii,"Oy Mola TV di Indihome nape tak bisa dibuka, a...",mola indihome nape tak bisa dibuka appsnya il...,mola indihome nape tak bisa dibuka appsnya il...,"[, mola, indihome, nape, tak, bisa, dibuka, ap...","[, mola, indihome, nape, dibuka, appsnya, ilang]",mola indihome nape dibuka appsnya ilang


In [47]:
###########################SAVE PREPRO###########################
#################################################################

data1.dropna()
# data1['Data Teks_Stemming']=data1['Data Teks_Stemming'].str.replace(' ', '')
data1.drop_duplicates(subset='Data Teks_Stemming',
                      keep= 'first',inplace=True)
data1.to_csv(Data_prepro, index=False, header=True)
# data2 = pd.read_csv(Data_prepro, sep=',').dropna(subset=['Data Teks_Stemming'])
data1.head(5)

,Unnamed: 0,Unnamed: 0.1,Tanggal,User ID,Data Teks,Data Teks_CaseFolding,Data Teks_clean_alay,Data Teks_Token,Data Teks_Stopword,Data Teks_Stemming
0,12473,12473,2021-10-27 07:31:26+00:00,arifsaadil,"@IndiHomeCare Sudah dilakukan sesuai DM, belum...",sudah dilakukan sesuai belum bisa,sudah dilakukan sesuai belum bisa,"[sudah, dilakukan, sesuai, belum, bisa]",[sesuai],sesuai
1,9030,9030,2021-10-11 10:42:41+00:00,tojizru,@chifoeyuu P @IndiHome,,,[],[],
2,8294,8294,2021-10-30 05:33:15+00:00,IndiHomeCare,@DilaPong Kak Dila. Uci cek updatenya dijadwal...,kak dila uci cek updatenya dijadwalkan kembali...,kak dila uci periksa updatenya dijadwalkan ke...,"[kak, dila, uci, periksa, updatenya, dijadwalk...","[dila, uci, periksa, updatenya, dijadwalkan, e...",dila uci periksa updatenya dijadwalkan esok ja...
3,9506,9506,2021-10-09 12:36:33+00:00,lalafindyou_,@IndiHome ni kapan ya wifinya bener. Aku mau n...,kapan wifinya bener aku mau nugas hikdddd pad...,kapan wifinya benar aku mau nugas hikdddd pa...,"[, kapan, wifinya, benar, aku, mau, nugas, hik...","[, wifinya, nugas, hikdddd, dah, bayar]",wifinya nugas hikdddd dah bayar
4,3036,3036,2021-10-23 11:43:16+00:00,Razyajii,"Oy Mola TV di Indihome nape tak bisa dibuka, a...",mola indihome nape tak bisa dibuka appsnya il...,mola indihome nape tak bisa dibuka appsnya il...,"[, mola, indihome, nape, tak, bisa, dibuka, ap...","[, mola, indihome, nape, dibuka, appsnya, ilang]",mola indihome nape dibuka appsnya ilang
